In [1]:
'''
env : yh_Diffuser
'''
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
from PIL import Image
from datasets import Dataset
from torchvision import transforms
import torch
import pickle
from torch import nn
import glob

from dataclasses import dataclass
from scipy.special import kl_div
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from math import pi
# from metrics import CPC, JSD_ODflow
from model import DeepGravity
mseloss = nn.MSELoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
import os
os.environ["HF_DATASETS_OFFLINE"] = "1"

cuda


In [2]:
@dataclass
class TrainingConfig:
    output_dir = "/mnt/data/chunhuachen/UMWIFI_TOOL/temp/"
config = TrainingConfig()
test_dir = os.path.join(config.output_dir, "split")
os.makedirs(test_dir, exist_ok=True)

In [3]:
import torch
import numpy as np

class GravityDataset(torch.utils.data.Dataset):
    def __init__(self, paths, labels):
        self.paths = paths
        self.labels = labels
        # self.days_per_month = [np.load(path, mmap_mode='r').shape[0] for path in paths]
        # self.days_per_month = [31,28,31,30,31,30,31,31,30,31,30,31,
        #                        31,28,31,30,31,30,31,31,30,31,30,31,
        #                        28,31,30,31,30,31,31,30,31,30,31,]
        self.days_per_month = [31]
        self.max = 157

    def __len__(self):
        return sum(day * 54 * 53 * 144 for day in self.days_per_month)

    def __getitem__(self, idx):
        # 计算.npy文件的索引
        path_idx = idx // (144*54*53)
        
        data = np.load(self.paths[path_idx], mmap_mode='r')
        # 计算OD pair和时间段的索引
        time_frame_idx = idx % 144
        od_idx = idx % (54*53)
        i = od_idx // 53
        j = od_idx % 53
        if j >= i:
            j += 1
        od_flow = data[time_frame_idx, i, j]/self.max

        time_idx = idx % 144
        # time_idx = idx // (53*54)

        # 获取OD pair的特征
        # features = data[idx // (54*53*144), i, j]
        # 计算流量的占比
        # fraction = features / np.sum(data[idx // (54*53*144), i, :], axis=-1)
        # odflow_sum = np.sum(data[time_frame_idx, i, :])
        # od_sum = 1 if odflow_sum <= 0 else odflow_sum
        # od_sum = 1 if np.sum(data[idx // (54*53*144), i, :]) <= 0 else np.sum(data[idx // (54*53*144), i, :])
        # fraction = features / od_sum
        # 获取标签并添加新的元素
        label = self.labels[(path_idx + 31+28) * 144 + time_idx].copy()
        label.extend([171+i, 226+j])
        # label = torch.stack(label)
        # return features, fraction, label
        return od_flow, label
    
# paths = []
# for i in range(1, 5):  # 对于1到4位的数字
#     pattern = f'/mnt/data/chunhuachen/UMWIFI_TOOL/data/UMWIFI_FINAL_SET/UMWIFI_202?/TRACE/month_??_trace_build_low10_up500_devices{"[0-9]"*i}_UserCentric_diagonal_insert_flow_outnode.npy'
#     paths.extend(glob.glob(pattern))
# paths.sort()
# data = [np.load(npy_dir, mmap_mode='r') for npy_dir in paths]
pattern = test_dir + "/month_02_day_??.npy" 
paths = glob.glob(pattern)
paths.sort()
with open('/home/dingqiyang/chunhuachen/umwifi/diffusers/jupyter_test/label_21_22_23_year.pkl', 'rb') as f:
    labels = pickle.load(f)
dataset = GravityDataset(paths,labels)
batchsize = 1024
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchsize,shuffle=True)

In [4]:
len(paths)

31

In [5]:
# 定义模型
config = batchsize
model = DeepGravity(config).to(device)
# # 定义损失函数
# criterion = torch.nn.CrossEntropyLoss()
# # 定义优化器
# optimizer = torch.optim.RMSprop(model.parameters(), lr=5e-6, momentum=0.9)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# 定义训练周期数
epochs = 5
from tqdm import tqdm
# 开始训练
for epoch in range(epochs):
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
    for i, (od_flow, label) in enumerate(pbar):
        od_flow = od_flow.float()
        od_flow = od_flow.to(device)
        label = torch.stack(label,axis=1).to(device)
        outputs = model(label)
        # print(outputs)
        loss = criterion(outputs.squeeze(), od_flow)
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 5, Loss: 1.090844762074994e-05: 100%|██████████| 12477/12477 [32:41<00:00,  6.36it/s] 


In [79]:
torch.save(model,'./trained_model/DGM_model.pth')

In [6]:
model.eval()

DeepGravity(
  (embed): Embedding(280, 128)
  (linear_in): Linear(in_features=1152, out_features=64, bias=True)
  (linears): ModuleList(
    (0-4): 5 x Linear(in_features=64, out_features=64, bias=True)
  )
  (linear_out): Linear(in_features=64, out_features=1, bias=True)
)

In [15]:
# label_21_22_23_year_date.pkl是有误版本，label_21_22_23_year_date_update.pkl是更新版本
with open('/home/dingqiyang/chunhuachen/umwifi/diffusers/jupyter_test/label_21_22_23_year_date_update.pkl', 'rb') as f:
    picked_labels_index = pickle.load(f)
tuple_of_lists = [tuple(x[1:]) for x in picked_labels_index]
indices_dict = {}
for i, sublist in enumerate(tuple_of_lists):
    sublist_tuple = tuple(sublist)
    if sublist_tuple not in indices_dict:
        indices_dict[sublist_tuple] = [i]
    else:
        indices_dict[sublist_tuple].append(i)
#monday ==> Mon, no day-off, no weather, offline
regular_index = indices_dict[(144, 151,156,160,165, 168)] 
#saturday ==> Sat, no day-off, no weather, offline
saturday_index = indices_dict[(149, 151,156,160,165, 168)] 
#hybrid ==> Fri, no day-off, no weather, HYBRID
hybrid_index = indices_dict[(148, 151,156,160,165, 169)] 
#recess ==> Thurs, RECESS, no weather, offline
recess_index = indices_dict[(147, 155,156,160,165, 168)] 
#typhoon ==> Sunday,no vacation, NO.8!!!, offline
tropical_8_index = indices_dict[(150, 151, 159, 164, 167, 168)] 
#black ==> Tues, examination, no typhoon, BLACK, offline
black_rain_index = indices_dict[(145, 152, 156, 163, 166, 168)]
#suspended ==> Sunday,no day-off,no typhoon,no raninstorm, THUNDERSTORM,SUSPENDED
suspended_index = indices_dict[(150, 151, 156, 160, 166, 171)]
#examination ==> Monday, EXAMINATION, THUNDER,normal [143,150]
examination_thunder_index = indices_dict[(144, 152, 156, 160, 166, 168)]
#hybrid holiday ==> Wed,HOLIDAY,HYBRID [264]
hybrid_holiday_index = indices_dict[(146, 153, 156, 160, 165, 169)]
index_name = ['regular','saturday','hybrid', 'recess','suspended','tropical_8', 'black',"examination_thunder","hybrid_holiday"]
index_set_all = [regular_index,saturday_index,hybrid_index,recess_index,suspended_index,tropical_8_index,black_rain_index,examination_thunder_index,hybrid_holiday_index]

@dataclass
class TrainingConfig:
    output_dir = "/home/dingqiyang/chunhuachen/umwifi/DGM"
config = TrainingConfig()
eval_dir = os.path.join(config.output_dir, "evaluation")
os.makedirs(test_dir, exist_ok=True)

In [14]:
paths = []
for i in range(1, 5):  # 对于1到4位的数字
    pattern = f'/mnt/data/chunhuachen/UMWIFI_TOOL/data/UMWIFI_FINAL_SET/UMWIFI_202?/TRACE/month_??_trace_build_low10_up500_devices{"[0-9]"*i}_UserCentric_diagonal_insert_flow_outnode.npy'
    paths.extend(glob.glob(pattern))
paths.sort()
data = [np.load(npy_dir, mmap_mode='r') for npy_dir in paths]

with open('/home/dingqiyang/chunhuachen/umwifi/diffusers/jupyter_test/label_21_22_23_year.pkl', 'rb') as f:
    labels = pickle.load(f)

def my_gen():
    label_count = 0
    for npy in data:
        # img = Image.fromarray(data[i][0].astype(np.uint8),"L")
        for i in range(npy.shape[0]):        
            img = npy[i].transpose(2,0,1)
            label = labels[label_count]
            label_count += 1
            yield {"image": img,"label":label}
dataset_eval = Dataset.from_generator(my_gen)

preprocess = transforms.Compose(
    [
        # transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

def transform(examples):
    images = [preprocess(torch.from_numpy(np.array(image))).float() for image in examples["image"]]
    labels = [torch.tensor(label) for label in examples["label"]]
    return {"images": images,"labels":labels}

dataset_eval.set_transform(transform)

Found cached dataset generator (/home/dingqiyang/.cache/huggingface/datasets/generator/default-cb3aa627cd75d403/0.0.0)


In [64]:
def generate_matrix(model, index_set, filename):
    # 初始化一个四维张量来存储结果
    time_slices=144
    matrix_size=54
    result = torch.zeros((time_slices, 1, matrix_size, matrix_size))
    # 遍历每个时间切片
    for t in range(time_slices):
        # 创建一个空的输入列表
        inputs = []
        # 创建一个空的索引列表
        indices = []
        input = dataset_eval[index_set[0]*144+t]['labels']
        # 遍历矩阵的每个元素
        for i in range(matrix_size):
            for j in range(matrix_size):
                # 对角线元素不生成，全为0
                if i != j:
                    # 创建一个新的输入，包含原始输入和当前的行列索引
                    new_input = torch.cat((input, torch.tensor([i+172, j+226]))).to(device)
                    # 将新的输入添加到输入列表
                    inputs.append(new_input)
                    # 将当前的行列索引添加到索引列表
                    indices.append((i, j))

        # 将输入列表转换为张量
        inputs = torch.stack(inputs)

        # 使用模型生成新的值，禁用梯度计算
        with torch.no_grad():
            outputs = model(inputs)

        # 将输出的值匹配到结果矩阵的非对角线元素
        for k, (i, j) in enumerate(indices):
            result[t, 0, i, j] = outputs[k]

    result = result*157
    torch.save(result, eval_dir + f'/{filename}.pth')



In [65]:
index_name = ['regular','saturday','hybrid', 'recess','suspended','tropical_8', 'black',"examination_thunder","hybrid_holiday"]
index_set_all = [regular_index,saturday_index,hybrid_index,recess_index,suspended_index,tropical_8_index,black_rain_index,examination_thunder_index,hybrid_holiday_index]
for i, index_set in enumerate(index_set_all):
    generate_matrix(model, index_set, index_name[i])
    print(index_name[i])

regular
saturday
hybrid
recess
suspended
tropical_8
black
examination_thunder
hybrid_holiday


In [76]:
import csv
from metrics import CPC, JSD_ODflow
mseloss = nn.MSELoss()
def evaluate(original,images):
    CPC_list_diag = []
    JSD_list_diag = []
    RMSE_list_diag = []
    NRMSE_list_diag = [] 
    CPC_list_OD = []
    JSD_list_OD = []
    RMSE_list_OD = []
    NRMSE_list_OD = [] 
    for j in range(images.shape[0]):
        diagonal_gen = (images[j,0,0:53,0:53].diag()*1786)
        diagonal_ori = (original[j,0,0:53,0:53].diag()*1786)
        CPC_diag = CPC(diagonal_gen,diagonal_ori)
        JSD_diag = JSD_ODflow(diagonal_gen,diagonal_ori)
        rmse_diag = torch.sqrt(mseloss(diagonal_gen,diagonal_ori))
        nrmse_diag = rmse_diag/1786
        CPC_list_diag.append(CPC_diag)
        JSD_list_diag.append(JSD_diag)
        RMSE_list_diag.append(rmse_diag)
        NRMSE_list_diag.append(nrmse_diag)
        # 计算其余值的CPC，JSD和RMSE
        OD_gen = (images[j,0,:,:].clone())#失误，在生成的时候就×过了
        OD_ori = (original[j,0,:,:].clone())*530
        OD_gen.fill_diagonal_(0)
        OD_ori.fill_diagonal_(0)
        CPC_OD = CPC(OD_gen, OD_ori)
        JSD_OD = JSD_ODflow(OD_gen, OD_ori)
        rmse_OD = torch.sqrt(mseloss(OD_gen, OD_ori))
        nrmse_OD = rmse_OD/530
        CPC_list_OD.append(CPC_OD)
        JSD_list_OD.append(JSD_OD)
        RMSE_list_OD.append(rmse_OD)
        NRMSE_list_OD.append(nrmse_OD)
# print(len(CPC_list_diag))
    return CPC_list_diag, JSD_list_diag, RMSE_list_diag, NRMSE_list_diag, CPC_list_OD, JSD_list_OD, RMSE_list_OD, NRMSE_list_OD

def index_set_evaluate(index_set, filename):
    metrics = ["cpc_flow", "jsd_flow", "rmse_flow", "nrmse_flow", "cpc_od", "jsd_od", "rmse_od", "nrmse_od"]
    results = {metric + "_all": [] for metric in metrics}


    images = torch.load(eval_dir + f'/{filename}.pth')


    for i, index in enumerate(index_set):
        day = torch.stack(dataset_eval[index * 144 : index * 144 + 144]['images'])
        day = day * 0.5 + 0.5
        metrics_values = evaluate(day, images.cpu())
        for metric, value in zip(metrics, metrics_values):
            results[metric + "_all"].append(value)

    print(filename, "Index length", len(index_set))
    results_avg = {}
    for metric in metrics:
        # avg = np.mean(results[metric + "_all"], axis=0).mean()
        avg = np.mean(results[metric + "_all"])
        # print(metric + ":", avg)
        results_avg[metric + "_avg"] = avg

    # print(filename, "Index length", len(index_set), ", ".join(map(str, [results_avg[metric + "_avg"] for metric in metrics])))
    return results_avg

In [77]:
pipeline = []
metrics = ["cpc_flow", "jsd_flow", "rmse_flow", "nrmse_flow", "cpc_od", "jsd_od", "rmse_od", "nrmse_od"]
all_results = {metric + "_all_set_avg": [] for metric in metrics}
with open(f'./evaluation/results_DGM.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([metric + "_avg" for metric in metrics])  # 写入表头
    for i, index_set in enumerate(index_set_all):
            results = index_set_evaluate(index_set, index_name[i])
            writer.writerow([results[metric + "_avg"] for metric in metrics])
            writer.writerow([])
            writer.writerow([])
            writer.writerow([])
            writer.writerow([])
            writer.writerow([])
            writer.writerow([])
            for metric in metrics:
                all_results[metric + "_all_set_avg"].append(results[metric+'_avg'])
    all_results_avg = {metric: np.mean(values) for metric, values in all_results.items()}
    writer.writerow([all_results_avg[metric + "_all_set_avg"] for metric in metrics])

regular Index length 31
saturday Index length 36
hybrid Index length 30
recess Index length 18
suspended Index length 1
tropical_8 Index length 1
black Index length 1
examination_thunder Index length 2
hybrid_holiday Index length 1
